# Gravity Drain Tank Modeling

In [1]:
%matplotlib widget

In [2]:
from pysketcher import *

In [3]:
from IPython.display import HTML, SVG, display, clear_output

In [4]:
from ipywidgets import interact, FloatSlider, Button

In [5]:
from math import pi
import numpy as np
from ipywidgets import VBox, FloatSlider, Output, Button, Label

In [6]:
import numpy as np
from scipy.integrate import odeint
import time

# Experiment, Simulation (x10)

In [7]:
myfig={}
sketch = Sketch(myfig)

In [8]:
sketch.file2Sketch("drainingtank.yml")

True

In [9]:
def tank(h):
    sketch.container['h'] = h
    sketch.refresh('interieur')
    sketch.refresh('contenu')
    sketch.refresh('hc')
    sketch.refresh('V')
    sketch.refresh('Y')
    sketch.refresh('jet')
    sketch.refresh('dim')
    sketch.refresh('tank')
    drawing_tool.erase()
    sketch.container['tank'].draw()
    display(SVG(Sketch.matplotlib2SVG()))

In [10]:
w = FloatSlider(min=sketch.container['d_o']*1.2, max=sketch.container['H'], step=0.1, value=sketch.container['h'])
b = Button(description="Simulate")
i = interact(tank, h=w)
display(i,b)

interactive(children=(FloatSlider(value=1.5, description='h', max=2.0, min=0.12), Output()), _dom_classes=('wi…

<function __main__.tank(h)>

Button(description='Simulate', style=ButtonStyle())

## Torricelli's law

Torricelli's law states the velocity of an incompressible liquid stream exiting a liquid tank at level $h$ below the surface is 

$$v = \sqrt{2gh}$$ 

This is the same velocity as an object dropped from a height $h$. The derivation is straightforward. From Bernoulli's principle,

$$\frac{v^2}{2} + gh + \frac{P}{\rho} = \text{constant}$$

Applying this principle, compare a drop of water just below the surface of the water at distance $h$ above the exit, to a drop of water exiting the tank

$$gh + \frac{P_{atm}}{\rho} = \frac{v^2}{2} + \frac{P_{atm}}{\rho}$$

$$\implies v^2 = 2gh$$
$$\implies v = \sqrt{2gh}$$

Torricelli's law is an approximation that doesn't account for the effects of fluid viscosity, the specific flow geometry near the exit, or other flow non-idealities. Nevertheless it is a useful first approximation for flow from a tank.

# Mass Balance for Tank with Constant Cross-Sectional Area

For a tank with constant cross-sectional area, such as a cylindrical or rectangular tank, the liquid height is described by a differential equation

$$A\frac{dh}{dt} = q_{in}(t) - q_{out}(t)$$

where $$q_{out}$$ is a function of liquid height. Torricelli's law tells the outlet flow from the tank is proportional to square root of the liquid height

$$ q_{out}(h) = C_v\sqrt{h} $$

Dividing by area we obtain a nonlinear ordinary differential equation 

$$ \frac{dh}{dt} = - \frac{C_V}{A}\sqrt{h} + \frac{1}{A}q_{in}(t) $$

in our standard form where the LHS derivative appears with a constant coefficient of 1.

# Equation Solving

## Construction parameters

In [11]:
A   = pi * sketch.container['l']**2 / 4              # Tank area [meter^2]
h0 = sketch.container['h']-sketch.container['d_o']   # tank initial height
Cv  = 0.1/60                                         # Outlet valve constant [cubic meters/sec/meter^1/2]

In [12]:
print(h0)

1.4


## Physical Constants

In [13]:
g = sketch.container['g']                         # gravity constant

## Time Horizon

In [14]:
horizon = 10 * 60   # simulation period

## System Solving

In [15]:
# inlet flow rate in cubic meters/sec
def qin(t):
    return 0.0

def deriv(h,t):
    return qin(t)/A - Cv*np.sqrt(np.abs(h))/A

In [16]:
def solver():
    global D,t,h,v0
    D = 2 * np.sqrt(A/pi)
    IC = [h0]
    t = np.linspace(0,horizon,101)
    h = odeint(deriv,IC,t)
    v0 = np.sqrt(2*g*np.abs(h))

In [17]:
solver()

## Animating Gravity Drain Tank

In [18]:
def simulation(b):
    global h, sketch, w
    for hin in [hauteur for hauteur in h if hauteur > sketch.container['d_o']]:
        w.value = hin
        time.sleep(0.1)

In [19]:
b.on_click(simulation)